In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

c:\Users\Arjun Sarkar\Anaconda\envs\llm\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
class AgentState(TypedDict):

    number1: int
    number2: int
    number3: int
    number4: int
    operation1: str
    operation2: str
    finalnumber1: int
    finalnumber2: int

In [ ]:
def adder1(state: AgentState) -> AgentState:
    """This node adds 2 numbers"""

    state['finalnumber1'] = state['number1'] + state['number2']
    return state

def adder2(state: AgentState) -> AgentState:
    """This node adds 2 numbers"""

    state['finalnumber2'] = state['number3'] + state['number4']
    return state

def subtractor1(state: AgentState) -> AgentState:
    """This node subtracts 2 numbers"""

    state['finalnumber1'] = state['number1'] - state['number2']
    return state

def subtractor2(state: AgentState) -> AgentState:
    """This node subtracts 2 numbers"""

    state['finalnumber2'] = state['number2'] - state['number3']
    return state

def decide_next_node1(state: AgentState) -> AgentState:

    if state["operation1"] == "+":
        return "add1"
    
    elif state["operation1"] == "-":
        return "sub1"
    
def decide_next_node2(state: AgentState) -> AgentState:

    if state["operation2"] == "+":
        return "add2"
    
    elif state["operation2"] == "-":
        return "sub2"

In [12]:
graph = StateGraph(AgentState)

graph.add_node("add1", adder1)
graph.add_node("sub1", subtractor1)
graph.add_node("add2", adder2)
graph.add_node("sub2", subtractor2)
graph.add_node("router1", lambda state:state)
graph.add_node("router2", lambda state:state)

graph.add_edge(START, "router1")
graph.add_conditional_edges(
    "router1",
    decide_next_node1,
    {
        "add1": "add1",
        "subt1": "sub1"
    }
)

graph.add_edge("add1", "router2")
graph.add_edge("sub1", "router2")

graph.add_conditional_edges(
    "router2",
    decide_next_node2,
    {
        "add2": "add2",
        "sub2": "sub2"
    }
)

graph.add_edge("add2", END)
graph.add_edge("sub2", END)

app = graph.compile()

In [13]:
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`